In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor
#from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path="/content/drive/My Drive/DS Training/Meterial/test_clubmahindra.csv"
path1="/content/drive/My Drive/DS Training/Meterial/train_clubmahindra.csv"

In [ ]:
%%time
test = pd.read_csv(path)
train = pd.read_csv(path1)

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train['booking_date'] = pd.to_datetime(train['booking_date'], format = '%d/%m/%y')
test['booking_date'] = pd.to_datetime(test['booking_date'], format = '%d/%m/%y')
train['checkin_date'] = pd.to_datetime(train['checkin_date'], format = '%d/%m/%y')
test['checkin_date'] = pd.to_datetime(test['checkin_date'], format = '%d/%m/%y')

train['checkout_date'] = pd.to_datetime(train['checkout_date'], format = '%d/%m/%y')
test['checkout_date'] = pd.to_datetime(test['checkout_date'], format = '%d/%m/%y')

train_index = list(train[train.checkin_date.dt.year ==2012].index)


In [ ]:
train.head(5)

In [ ]:
## There was one record where room nights was -45, changing the record to 7 as per difference between checkin and checkout date

train.loc[train.roomnights <0, 'roomnights'] = 7 

In [ ]:
##There is only one record in train which had reservationstatusid_code as D

ind = train[train.reservationstatusid_code == 'D'].index
train.drop(index = ind, axis =0, inplace = True)
train = train.reset_index(drop = True)

In [ ]:
## filling the missing values with 0 (state unknown)
train['state_code_residence'].fillna(value =8.0, inplace = True)
test['state_code_residence'].fillna(value =8.0, inplace = True)

## filling the missing values with 0 
train['season_holidayed_code'].fillna(value =2.0, inplace = True)
test['season_holidayed_code'].fillna(value =2.0, inplace = True)


In [ ]:
train.dtypes

In [ ]:
## Date features for Booking date

train['booking_year'] = train['booking_date'].dt.year
train['booking_month'] = train['booking_date'].dt.month
train['booking_Week'] = train['booking_date'].dt.week
train['booking_Day'] = train['booking_date'].dt.day
train['booking_weeknum'] = train['booking_date'].dt.weekofyear
train['booking_dayofweek'] = train['booking_date'].dt.dayofweek
train['booking_Dayofyear'] = train['booking_date'].dt.dayofyear

test['booking_year'] = test['booking_date'].dt.year
test['booking_month'] = test['booking_date'].dt.month
test['booking_Week'] = test['booking_date'].dt.week
test['booking_Day'] = test['booking_date'].dt.day
test['booking_weeknum'] = test['booking_date'].dt.weekofyear
test['booking_dayofweek'] = test['booking_date'].dt.dayofweek
test['booking_Dayofyear'] = test['booking_date'].dt.dayofyear

In [ ]:
## Total person travelling as sum of adult count and children count

train['total_nbr_ppl'] = train['numberofadults'] + train['numberofchildren']
test['total_nbr_ppl'] = test['numberofadults'] + test['numberofchildren']


In [ ]:
## Total person travelling as sum of adult count and children count

train['total_nbr_ppl'] = train['numberofadults'] + train['numberofchildren']
test['total_nbr_ppl'] = test['numberofadults'] + test['numberofchildren']

In [ ]:
## Date difference features between checkin and checkout date

train['prior_booking'] = (train['checkin_date'] - train['booking_date']).dt.days
test['prior_booking'] = (test['checkin_date'] - test['booking_date']).dt.days

train['length_of_stay'] = (train['checkout_date'] - train['checkin_date']).dt.days 
test['length_of_stay'] = (test['checkout_date'] - test['checkin_date']).dt.days

train['prior_booking_week'] = ((train['checkin_date'] - train['booking_date']).dt.days)/7
#train['prior_booking_week'] = train['prior_booking_week'].apply(lambda x: math.ceil(x))

test['prior_booking_week'] = ((test['checkin_date'] - test['booking_date']).dt.days)/7
#test['prior_booking_week'] = test['prior_booking_week'].apply(lambda x: math.ceil(x))


train['prior_booking_month'] = ((train['checkin_date'] - train['booking_date']).dt.days)/30
#train['prior_booking_month'] = train['prior_booking_month'].apply(lambda x: math.ceil(x))

test['prior_booking_month'] = ((test['checkin_date'] - test['booking_date']).dt.days)/30
#test['prior_booking_month'] = test['prior_booking_month'].apply(lambda x: math.ceil(x))

In [ ]:
## Date features for checkin date

train['checking_year'] = train['checkin_date'].dt.year
test['checking_year'] = test['checkin_date'].dt.year

train['checking_month'] = train['checkin_date'].dt.month
test['checking_month'] = test['checkin_date'].dt.month

train['checking_day'] = train['checkin_date'].dt.day
test['checking_day'] = test['checkin_date'].dt.day

#train['checking_weekday'] = train['checkin_date'].dt.weekday_name
#test['checking_weekday'] = test['checkin_date'].dt.weekday_name

train['is_year_end_checkin'] = train['checkin_date'].dt.is_year_end
test['is_year_end_checkin'] = test['checkin_date'].dt.is_year_end


train['is_year_start_checkin'] = train['checkin_date'].dt.is_year_start
test['is_year_start_checkin'] = test['checkin_date'].dt.is_year_start


train['is_month_end_checkin'] = train['checkin_date'].dt.is_month_end
test['is_month_end_checkin'] = test['checkin_date'].dt.is_month_end


train['is_month_start_checkin'] = train['checkin_date'].dt.is_month_start
test['is_month_start_checkin'] = test['checkin_date'].dt.is_month_start

In [ ]:
## imputing the rows where we have issues in prior booking with median prior booking day of 33 days

train.loc[train.prior_booking <0, 'prior_booking'] = 33


In [ ]:
nbr_booking_train = train.groupby(by = 'memberid').count()[['reservation_id']].reset_index()
nbr_booking_train.columns = ['memberid','nbr_booking']

train = train.merge(nbr_booking_train, on= 'memberid')

nbr_booking_test = test.groupby(by = 'memberid').count()[['reservation_id']].reset_index()
nbr_booking_test.columns = ['memberid','nbr_booking']

test = test.merge(nbr_booking_test, on= 'memberid')

nbr_booking_train = train.groupby(by = 'memberid').sum()[['length_of_stay']].reset_index()
nbr_booking_train.columns = ['memberid','total_stay_days']

train = train.merge(nbr_booking_train, on= 'memberid')

nbr_booking_test = test.groupby(by = 'memberid').sum()[['length_of_stay']].reset_index()
nbr_booking_test.columns = ['memberid','total_stay_days']

test = test.merge(nbr_booking_test, on= 'memberid')


nbr_booking_train = train.groupby(by = 'memberid').mean()[['length_of_stay']].reset_index()
nbr_booking_train.columns = ['memberid','avg_stay_days']

train = train.merge(nbr_booking_train, on= 'memberid')

nbr_booking_test = test.groupby(by = 'memberid').mean()[['length_of_stay']].reset_index()
nbr_booking_test.columns = ['memberid','avg_stay_days']

test = test.merge(nbr_booking_test, on= 'memberid')


nbr_booking_train = train.groupby(by = 'memberid').mean()[['numberofadults']].reset_index()
nbr_booking_train.columns = ['memberid','avg_numberofadults']

train = train.merge(nbr_booking_train, on= 'memberid')

nbr_booking_test = test.groupby(by = 'memberid').mean()[['numberofadults']].reset_index()
nbr_booking_test.columns = ['memberid','avg_numberofadults']

test = test.merge(nbr_booking_test, on= 'memberid')



nbr_booking_train = train.groupby(by = 'memberid').mean()[['total_nbr_ppl']].reset_index()
nbr_booking_train.columns = ['memberid','avg_total_nbr_ppl']

train = train.merge(nbr_booking_train, on= 'memberid')

nbr_booking_test = test.groupby(by = 'memberid').mean()[['total_nbr_ppl']].reset_index()
nbr_booking_test.columns = ['memberid','avg_total_nbr_ppl']

test = test.merge(nbr_booking_test, on= 'memberid')

In [ ]:

## delete reservation id columns

train.drop(columns='reservation_id', axis =1, inplace= True)
test.drop(columns='reservation_id', axis =1, inplace= True)

## delete date columns

train.drop(columns=['booking_date','checkin_date','checkout_date'], axis =1, inplace= True)
test.drop(columns=['booking_date','checkin_date','checkout_date'], axis =1, inplace= True)

## delete member id columns

train.drop(columns='memberid', axis =1, inplace= True)
test.drop(columns='memberid', axis =1, inplace= True)




In [ ]:
test=test.drop(columns=['amount_spent_per_room_night_scaled'], axis =1, inplace= True)
train=train.drop(columns=['amount_spent_per_room_night_scaled'], axis =1, inplace= True)

In [ ]:
## Created data for baseline linear model
df_train = pd.get_dummies(train)
#df_test = pd.get_dummies(test)

In [ ]:
train.head(20)

In [ ]:
#Importing the simple LinearRegression
from sklearn.linear_model import LinearRegression
#Importing the simple LinearRegression
reg = LinearRegression()

In [ ]:
#Fitting the data to the regression
reg.fit(df_train, df_test)

# New Section